# Useful Applications

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import datetime as dt
import math

from jupyterthemes import jtplot
jtplot.style(theme='onedork', figsize=(15, 9))

## Roll's model to estimate spread (1984)

- __[Trading costs - The Roll Spread Estimator](https://ba-odegaard.no/teach/notes/liquidity_estimators/roll_spread_estimator/roll_lectures.pdf)__
- __[A Simple Implicit Measure of the Effective Bid-Ask Spread in an Efficient Market](https://onlinelibrary.wiley.com/doi/full/10.1111/j.1540-6261.1984.tb03897.x)__

In [6]:
df = pd.read_csv('../../data/IBM_2013-09-01_2013-11-11_1d.csv', index_col=0, parse_dates=True)
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2013-09-03,117.398367,117.839496,116.682332,117.609344,3647611,0.0,0.0
2013-09-04,117.366468,117.756444,116.554525,117.078766,2717403,0.0,0.0
2013-09-05,117.219388,118.274267,117.040379,117.730843,2999510,0.0,0.0
2013-09-06,118.050496,118.267872,116.771854,117.014801,3037061,0.0,0.0
2013-09-09,117.430359,118.587533,117.193810,118.261475,3155991,0.0,0.0


In [7]:
close = df['Close'].to_frame()
close.head()

,Close
Date,
2013-09-03,117.609344
2013-09-04,117.078766
2013-09-05,117.730843
2013-09-06,117.014801
2013-09-09,118.261475


In [8]:
close['Diff'] = df['Close'].diff()
close.head()

,Close,Diff
Date,,
2013-09-03,117.609344,NaN
2013-09-04,117.078766,-0.530579
2013-09-05,117.730843,0.652077
2013-09-06,117.014801,-0.716042
2013-09-09,118.261475,1.246674


In [69]:
_cov = np.cov(close['Diff'][1:-1], close['Diff'][2:])
_cov[0, 1]

-0.14103707800980902

In [68]:
np.corrcoef(close['Diff'][1:-1], close['Diff'][2:])[0,1]

-0.05134942727349844

In [72]:
if _cov[0, 1] < 0:
    print(f'Roll\'s Spread for IBM: {round(2*np.sqrt(-_cov[0,1]), 3)}')
else:
    print(f'Roll\'s Spread for IBM: {round(2*np.sqrt(_cov[0,1]), 3)}')

Roll's Spread for IBM: 0.751


In [ ]:
ibm_wo_adjustment = yf.download('IBM', dt.datetime(2013, 9, 11), dt.datetime(2013, 11, 11), '1d')

In [ ]:
def rolls_spread(ticker, start=dt.datetime(2002, 1, 1), end=dt.datetime(2022, 12, 31), interval='1d'):
    pass